<p><strong><font size="6">WALOUS - Fusion LC classifications results</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys

** Add folder with SCR provided belong to this notebook**

In [3]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [4]:
run ../SRC/config.py

In [5]:
print config_parameters

{'permanent_mapset': 'PERMANENT', 'locationepsg': '31370', 'outputfolder': '../../Results', 'gisdb': '../../GRASSDATA', 'PYTHONLIB': '/usr/lib/python2.7', 'njobs': 8, 'GISBASE': '/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu'}


In [6]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [7]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/script:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2081 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/2010,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/2010 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LI

** GRASS GIS Python libraries **

In [8]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

** Other functions**

In [9]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon

**Create list for temporary layers management**

In [10]:
# Create list for storing name of intermediates layers
TMP_rast = []
TMP_vect = []

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Preprocessing

## Create general GRASSDATA

In [11]:
# Check if the GRASS GIS database exists and create it if not
check_gisdb(config_parameters["gisdb"])
# Check if the location exists and create it if not, with the CRS defined by the epsg code 
check_location(config_parameters["gisdb"],"general",config_parameters["locationepsg"])
# Change the current working GRASS GIS session mapset
working_mapset(config_parameters["gisdb"],"general",config_parameters["permanent_mapset"])

GRASSDATA folder already exist
Location 'general' already exist
You are now working in mapset 'general/PERMANENT'


## Get a list of 'timeclass'

In [28]:
# Import GPKG with all tiles
tile_file = os.path.join(data['obia_folder'],'tuiles_herve.gpkg')
gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='tuiles')

# Get a list with unique 'cutline' of in the tile layer
timeclass_list = list(gscript.parse_command('v.db.select', flags='c', map='tuiles', columns='timeclass'))
print "There are %s different 'timeclass' value (%s)"%(len(timeclass_list),','.join(timeclass_list))

There are 5 different 'timeclass' value (146555,147343,146892,146894,147324)


## Create multiple mapset for parallel computing

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [23]:
def launch_mapset(timeclass):
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    check_location(config_parameters["gisdb"],timeclass,config_parameters["locationepsg"])
    # Change the current working GRASS GIS session mapset
    working_mapset(config_parameters["gisdb"],timeclass,config_parameters["permanent_mapset"])

In [57]:
def import_vector_tile(timeclass):
    """ Function that import the tiles only of a specific 'timeclass' """
    condition = "timeclass=%s"%timeclass
    try:
        gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='cutline', where=condition) 
        #gscript.run_command('v.external', overwrite=True, input=tile_file, output='timeclass_%s'%timeclass, where=condition) 
        print "Tiles with condition 'timeclass=%s' has been imported."%timeclass
    except:
        print "ERROR: Tiles has not been imported. Please check for problem."%timeclass

In [48]:
def import_raster_segments(timeclass):
    """ Function that import the segmentation layer """
    input_file = os.path.join(data["obia_folder"],"SEGMENTS","segs_herve_%s.tif"%timeclass)
    try:
        gscript.run_command('r.in.gdal', flags='r', overwrite=True, input=input_file, output='segments')
        #gscript.run_command('r.external', overwrite=True, input=input_file, output='segments')
        print "Segmentation layer has been imported."
    except:
        print "ERROR: Segmentation layer has not been imported. Please check for problem."

In [46]:
def import_raster_obiaclass(timeclass):
    """ Function that import the OBIA classification """
    input_file = os.path.join(data["obia_folder"],"RASTER","classification_herve_%s.tif"%timeclass)
    try:
        gscript.run_command('r.in.gdal', flags='r', overwrite=True, input=input_file, output='obia_clas')
        #gscript.run_command('r.external', overwrite=True, input=input_file, output='obia_clas')
        print "OBIA classification layer has been imported."
    except:
        print "ERROR: OBIA classification layer has not been imported. Please check for problem."

In [47]:
def import_raster_pixelclass(timeclass):
    """ Function that import the Pixel-based classification """
    input_file = data['pix_orth']
    try:
        gscript.run_command('r.in.gdal', flags='r', overwrite=True, input=input_file, output='pixel_clas')
        print "Pixel-based classification layer has been imported."
    except:
        print "ERROR: Pixel-based classification layer has not been imported. Please check for problem."

In [78]:
def worker(timeclass):
    launch_mapset(timeclass) #Start GRASS session on a new mapset
    import_vector_tile(timeclass) #Import vector of cutlines with the same timeclass
    gscript.run_command('g.region', flags='s', vector='cutline')  #Define computational region
    import_raster_segments(timeclass) #Import raster layer with segments
    import_raster_obiaclass(timeclass) #Import raster layer with object-based classification
    import_raster_pixelclass(timeclass) #Import raster layer with pixel-based classification

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [77]:
for currentmapset in timeclass_list[1:2]:
    worker(currentmapset)

Location '147343' already exist
You are now working in mapset '147343/PERMANENT'
Tiles with condition 'timeclass=147343' has been imported.
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.


In [76]:
# Get a list with unique 'cat' of cutlines tile
cutlines_cats = list(gscript.parse_command('v.db.select', flags='c', map='cutline', columns='cat_'))
cutlines_cats

[u'7074', u'7048', u'7073']

In [ ]:
for cutline in cutlines_cats[0:1]:
    currentcutline
    

In [ ]:
v.extract --overwrite input=cutline@PERMANENT where=cat_=7848 output=tmp_cutline
g.region -a vector=test@PERMANENT
r.mask --overwrite vector=tmp_cutline@PERMANENT


## Get a list of tiles id from the file name

This list will be use in a further step of developement for distribute jobs on different nodes on HYDRA. 

In [ ]:
import glob  # Library needed to get the list of file in a folder
# Will return a list of path to each file with extension '*.gpkg.gz' in the folder 
files_paths = glob.glob(os.path.join(data['obia_folder'],'VECTEUR','*.gpkg.gz'))  

list_tile_id = []
for a in files_paths:
    file_name = os.path.split(a)[1]  # Keep only the file name with extension 
    list_tile_id.append(file_name.split('_')[1][len('tile'):])  # tileid should be on index 1 in this list

print "Obia results (vector) for %s tiles found in '%s'"%(len(list_tile_id),os.path.join(data['obia_folder'],'VECTEUR'))

## Launch GRASS GIS session that will work on the first tile

In [ ]:
# Define the id of the current tile
tile_id = list_tile_id[0]
# Define the name of the location
grasslocation = "WALOUS_%s"%tile_id

In [ ]:
# Check if the GRASS GIS database exists and create it if not
check_gisdb(config_parameters["gisdb"])
# Check if the location exists and create it if not, with the CRS defined by the epsg code 
check_location(config_parameters["gisdb"],grasslocation,config_parameters["locationepsg"])
# Change the current working GRASS GIS session mapset
working_mapset(config_parameters["gisdb"],grasslocation,config_parameters["permanent_mapset"])

## Import tile polygon and define computation region

**Unzip the geopackage file**

In [ ]:
files_paths[0]

# Import data / Preparation of data

In [ ]:
# Saving current time for processing time management
begintime_importdata=start_processing()

In [ ]:
import zipfile
path = os.path.join(data['obia_folder'],'VECTEUR','segs_tile%s_stats.gpkg.gz'%tile_id)

In [ ]:
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [ ]:
from sh import gunzip
gunzip(path)


## Import training samples (points)

In [ ]:
# Import point samples (only for the computational region)
#gscript.run_command('v.import', overwrite=True, input=data['admin'][1], output=data['admin'][0])

In [ ]:
## Print processing time
print_processing_time(begintime_processing ,"All processing terminate in ")

## Import pixel-based classification

## Import object-based classification

# Exports and cleaning mapset

## Export results

In [ ]:
# Output folder isochrones
outputfolder_isochrones = os.path.join(config_parameters['outputfolder'],"Isochrones")
# Check and create folder if needed
check_create_dir(outputfolder_isochrones)

In [ ]:
# Export isochrone layers as GeoPackage and attribute table as .csv
for isochrone in isochrone_layers:
    output_gpkg = os.path.join(outputfolder_isochrones,"%s.gpkg"%isochrone)
    output_csv = os.path.join(config_parameters['outputfolder'],"Prop_by_Isochrones_%s.csv"%isochrone[11:])
    gscript.run_command('v.out.ogr', flags='m', overwrite=True,
                        input=isochrone, output=output_gpkg, format="GPKG")
    gscript.run_command('v.out.ogr', flags='m', overwrite=True,
                        input=isochrone, output=output_csv, format="CSV")

## Remove intermediate layers

In [ ]:
# Remove all layers not needed anymore
for rast in TMP_rast:
    gscript.run_command('g.remove', flags='fb', type='raster', name=rast)

In [ ]:
# Remove all layers not needed anymore
for vect in TMP_vect:
    gscript.run_command('g.remove', flags='fb', type='vector', name=vect)

## Delete both mapsets

In [ ]:
import shutil

# Delete mapset folder in GRASSDATA
shutil.rmtree(config_parameters['gisdb'])